# Pulls image data from covid-chestxray-dataset
Pulls image data from https://github.com/ieee8023/covid-chestxray-dataset.git and outputs a zip folder containting the images and a csv file containing the image metadata

In [ ]:
!pip3 install wget==3.2 gitpython==3.1.14 ansible==3.1.0

In [ ]:
import wget
wget.download(
    'https://raw.githubusercontent.com/'
    'elyra-ai/component-library/master/claimed_utils.py'
)

In [ ]:
from ansible.module_utils.parsing.convert_bool import boolean
from claimed_utils import zipdir
import git
from io import BytesIO
from minio import Minio
import os
import shutil

In [ ]:
# @param output_filename
# @param metadata_filename
# @returns images.zip
# @returns metadata.csv

In [ ]:
output_filename = os.environ.get('output_filename', 'images.zip')
metadata_filename = os.environ.get('metadata_filename', 'metadata.csv')
checkpoint = boolean(os.environ.get('checkpoint', False))
checkpoint_ip = os.environ.get('checkpoint_ip')
checkpoint_user = os.environ.get('checkpoint_user', 'minio')
checkpoint_pass = os.environ.get('checkpoint_pass', 'minio123')
checkpoint_bucket = os.environ.get('checkpoint_bucket', 'checkpoint')

In [ ]:
exists = False

if checkpoint:
    client = Minio(checkpoint_ip,
                   checkpoint_user,
                   checkpoint_pass,
                   secure=False)

    objects = client.list_objects(checkpoint_bucket)
    asset_name = output_filename
    exists1 = False
    for obj in objects:
        if asset_name == obj.object_name:
            exists1 = True
            client.fget_object(checkpoint_bucket, asset_name, asset_name)
            break

    objects = client.list_objects(checkpoint_bucket)
    asset_name = metadata_filename
    exists2 = False
    for obj in objects:
        if asset_name == obj.object_name:
            exists2 = True
            client.fget_object(checkpoint_bucket, asset_name, asset_name)
            break

    exists = exists1 and exists

In [ ]:
if not exists:
    git.Git(".").clone("https://github.com/"
                       "ieee8023/covid-chestxray-dataset.git")
    shutil.move("covid-chestxray-dataset/images", ".")
    shutil.move("covid-chestxray-dataset/metadata.csv", ".")
    shutil.rmtree('covid-chestxray-dataset')
    zipdir('images.zip', 'images')

In [ ]:
if not exists:
    size = os.path.getsize(output_filename)
    with open(output_filename, 'rb') as fh:
        buf = BytesIO(fh.read())
        result = client.put_object(
            checkpoint_bucket, output_filename, buf, length=size
        )

    size = os.path.getsize(metadata_filename)
    with open(metadata_filename, 'rb') as fh:
        buf = BytesIO(fh.read())
        result = client.put_object(
            checkpoint_bucket, metadata_filename, buf, length=size
        )